In [ ]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 

### Support Functions

In [ ]:
def openAndReadImage(nameImg): #Operation to open 1 image
      image = Image.open(nameImg)
      image = np.array(image)
      return image

def iamgePreProcess(image): #pre-processing of the image
      width=image.shape[0]
      height=image.shape[1]
      channels=image.shape[2]
      flat_image = image.reshape(width* height,channels)
      return flat_image,width, height, channels

def check(pixel): # handle pixel when pixel > 255 or pixel < 0   
    if pixel <= 0: 
          return 0
    elif pixel >= 255: 
          return 255
    return pixel

def output(result,width,height,n_channel,name,input,save=False):
      result=result.reshape(width,height,n_channel) # reshape new image to original size
      if(save==False):
       display(Image.fromarray(result))
      else:
        output_file = name.split('.')[0]+input+"."+name.split('.')[1] # Generate file name according to input format
        Image.fromarray(result).save(output_file)

### Change the brightness of the image

In [ ]:
def adjustBrightness(image, brightness): # Adjust the brightness using the formula  
    for i in range(len(image)):
        R,G,B=image[i] # take RGB color to add brightness value 
        image[i]=check(R+brightness),check(G+brightness),check(B+brightness)    
    return image



In [ ]:

def test_Brightness(name,brightness,save):
    img=openAndReadImage(name)
    
    image,width,height,n_channel = iamgePreProcess(img)
    result=adjustBrightness(image,brightness)
    output(result,width,height,n_channel,name,"_brightness",save)
        

### Change the contrast

In [ ]:
def adjustContract(image,contrast): # Adjust the contrast using the formula  
    for i in range(len(image)):
        R,G,B=image[i]
        factor = (259 * (contrast + 255)) / (255 * (259 - contrast))
        image[i]=check(factor*(R-128)+128),check(factor*(G-128)+128),check(factor*(B-128)+128)
    return image

In [ ]:
def test_Contract(name,contract,save):
    img=openAndReadImage(name)
    image,width,height,n_channel = iamgePreProcess(img)
    result=adjustContract(image,contract)
    output(result,width,height,n_channel,name,"_contract",save)

### FLip Image

In [ ]:
def flip_Image(image,input):
    result = np.array(image)
    if input=='V':  # Vertical 
        return np.flip(result, axis=0)
    elif input=='H': # Horizontal
        return np.flip(result, axis=1)


In [ ]:
def test_Flip_Image(name,choose,save):
    img=openAndReadImage(name)
    result=flip_Image(img,choose)
    if(save==False):
        display(Image.fromarray(result))
    else:
        if(choose=="H"):
             output_file = name.split('.')[0]+"_Horizontal"+"."+name.split('.')[1]
        else:
             output_file = name.split('.')[0]+"_Vertical"+"."+name.split('.')[1]
        Image.fromarray(result).save(output_file)

### Gray Scale

In [ ]:
def changeGrayScale(image): # gray scale image using formula
    for i in range(len(image)):
        R,G,B=image[i]
        gray=check(R*0.2989)+check(G*0.5870)+check(B*0.1140) 
        image[i]=gray,gray,gray   
    return image

In [ ]:
def test_GrayScale(name,save):
    img=openAndReadImage(name)
    image,width,height,n_channel = iamgePreProcess(img)
    result=changeGrayScale(image)
    output(result,width,height,n_channel,name,"_grayscale",save)

### Plus 2 Image have same size

In [ ]:

def plus2Image(image1,image2,alpha):
    image01,width,height,n_channel = iamgePreProcess(image1)
    img01=changeGrayScale(image01)
    img01=img01.reshape(width,height,n_channel) # create gray iamge
    
    image02,width,height,n_channel = iamgePreProcess(image2)
    img02=changeGrayScale(image02)
    img02=img02.reshape(width,height,n_channel) # create gray iamge
    
    result = np.zeros(img01.shape,dtype=img01.dtype) # because the two pictures are the same size (Take image1 or image2) 
    result[:,:,:] = (alpha * img01[:,:,:]) + ((1-alpha) * img02[:,:,:])
    return result


In [ ]:
def test_Plus_2Image(name1,name2,alpha,save):
    img1=openAndReadImage(name1)
    img2=openAndReadImage(name2)
    
    result=plus2Image(img1, img2,alpha)
    if save==False:
        display(Image.fromarray(result))
    else:
        Image.fromarray(result).save("ResultPlus2Image.png")
        

### Blur Iamge Using Box Blur

In [ ]:
def box_blur(img,height,width):
    result  = img.copy()
    for x in range(0,width-1):
        for y in range(0,height-1):
            sum_pixels = (0, 0, 0)
            for pixel in [(x-1,y+1),(x,y+1),(x+1,y+1), (x-1,y),(x,y),(x+1,y),(x-1,y-1),(x,y-1),(x+1,y-1)]:
                sum_pixels=tuple(map(lambda i, j: i + j, sum_pixels,img[pixel]))
            result[x][y]=tuple(map(lambda i, j: i // j, sum_pixels, (9, 9, 9))) # assignment new color for new image have x ,y coordinate
    return result

In [ ]:
def test_Blur(name,save):
    img=openAndReadImage(name)
    _,width,height,n_channel = iamgePreProcess(img)
    result=box_blur(img,height,width)
    output(result,width,height,n_channel,name,"_Blur",save)

### Crop a photo with a circle frame

In [ ]:
def crop_Circle(name,save):
    img=Image.open(name)
    width,height=img.size
   
   
    center = (int(width/2), int(height/2)) # take center of circle
    radius = min(center[0], center[1], width-center[0], height-center[1]) # calculate radius 
    
    Y,X = np.ogrid[0:height, 0:width]
    mask = (X- (width/ 2)) ** 2 + (Y-(height / 2)) ** 2 <= radius**2 # circle equation
    
    # Create same size alpha layer with circle
    alpha = Image.new('L', [width,height],0) 
    npAlpha=np.array(alpha) # Convert alpha Image to numpy array
    npAlpha[~mask] = 0
    npAlpha[mask] = 255
    # Add alpha layer to RGB
    result=np.dstack((img,npAlpha))
    
    if save==False:
         display(Image.fromarray(result))
    else:
        output_file = name.split('.')[0]+"_cropCircle"+".png"
        Image.fromarray(result).save(output_file)
        



### The frame is 2 ellipses that cross each other

In [ ]:
def generate_ellipse(x, y, x_centre, y_centre, rotation, a, b):
        term1 = ((x - x_centre) * np.cos(rotation) + 
            (y - y_centre) * np.sin(rotation))**2
        term2 = ((x - x_centre) * np.sin(rotation) + 
            (y - y_centre) * np.cos(rotation))**2
        ellipse = ((term1 / a**2) + (term2 / b**2)) <= 1
        return ellipse  # True for points inside the ellipse
def crop_2_Ellipses_Cross(name,save):
    a,b = 0.9,0.9
    x0,y0=0,0
    img =Image.open(name)
    width,height=img.size
    
    # set up a coordinate system
    x = np.linspace(-1.2, 1.2, width)
    y = np.linspace(-1.2, 1.2, height)[:,
                                None]


    ellipse1 = generate_ellipse(x, y, x0, y0, 9.78, a, b) # create a 2D ellipse01
    ellipse2 = generate_ellipse(x, y, x0, y0, -9.78, a, b)# create a 2D ellipse02
   

    alpha = Image.new('L', [width,height],0) # Create same size alpha layer with ellipses
    npAlpha=np.array(alpha)# Convert alpha Image to numpy array
   # Create same size alpha layer with 2 ellipses cross
    npAlpha[ellipse1] = 255
    npAlpha[ellipse2] = 255 
    
    # Add alpha layer to RGB
    result=np.dstack((img,npAlpha))
    if save==False:
        
         display(Image.fromarray(result))
    else:
        output_file = name.split('.')[0]+"_crop2Ellipses"+".png"
        a=Image.fromarray(result)
        a.save(output_file)
        

### Main

In [ ]:
def main():
  name=str(input("Enter name of the image: "))
  print("1.Brightness of image")
  print("2.Contract of image")
  print("3.Grayscale of image")
  print("4.Flip of image")
  print("5.Plus 2 image")
  print("6.Blur image")
  print("7.Crop iamge circle")
  print("8.Crop iamge using 2 ellipses that cross each other")
  number=int(input("Enter choose :"))
  if(number==1):
      brightness=int(input("Enter brightness :"))
      print("--------------------------------RESULT--------------------------------")
      print("Brightness : ",brightness)
      test_Brightness(name,brightness,save=False)
  elif(number==2):
      contract=int(input("Enter contract :"))
      print("--------------------------------RESULT--------------------------------")
      print("Contract : ",contract)
      test_Contract(name,contract,save =False)
  elif(number==3):
      print("--------------------------------RESULT--------------------------------")
      test_GrayScale(name,save =False)
  elif(number==4):
      choose=str(input("Enter H (horizontal) or V (vertical) :"))
      print("--------------------------------RESULT--------------------------------")
      if(choose=='H'):
          print("Horizontal")
      else:
          print("Vertical")
      test_Flip_Image(name,choose,save =False)
  elif(number==5):
      name1=str(input("Enter name 01 of the image: "))
      name2=str(input("Enter name 02 of the image: "))
      alpha=float(input("Enter alpha :"))
      print("--------------------------------RESULT--------------------------------")
      test_Plus_2Image(name1,name2,alpha,save=False)
  elif(number==6):
      print("--------------------------------RESULT--------------------------------")
      test_Blur(name,save =False)
       
  elif(number==7):
      print("--------------------------------RESULT--------------------------------")
      crop_Circle(name,save=False)
       
  elif(number==8):
      print("--------------------------------RESULT--------------------------------")
      crop_2_Ellipses_Cross(name,save =False)
  elif(number==0):
      print("--------------------------------RESULT--------------------------------")
      print("Check Folder")
      brightness=50
      contract=128
      choose01='H'
      choose02='V'
      test_Brightness(name,brightness,save=True)                                               
      test_Contract(name,contract,save =True)
      test_GrayScale(name,save =True)
      test_Flip_Image(name,choose01,save =True)
      test_Flip_Image(name,choose02,save =True)
      test_Blur(name,save =True)
      crop_Circle(name,save=True)
      crop_2_Ellipses_Cross(name,save =True)
      name1=str(input("Enter name 01 of the image: "))
      name2=str(input("Enter name 02 of the image: "))
      alpha=float(input("Enter alpha :"))
      print("--------------------------------RESULT--------------------------------")
      test_Plus_2Image(name1,name2,alpha,save=True)
  else:
        print("Please again!")

In [ ]:
if __name__ == "__main__":
    
    main()
    